In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
sns.set_style("whitegrid")
import altair as alt
#alt.renderers.enable("notebook")

# Code for hiding seaborn warnings
import warnings
warnings.filterwarnings("ignore")

df_path = "C:\\FE\\autocar_dataset.csv"
#df_path2 = df_path + 'News_dataset.csv'
df = pd.read_csv(df_path)

df.head()

In [ ]:
bars = alt.Chart(df).mark_bar(size=50).encode(
    x=alt.X("Category"),
    y=alt.Y("count():Q", axis=alt.Axis(title='Number of articles')),
    tooltip=[alt.Tooltip('count()', title='Number of articles'), 'Category'],
    color='Category'

)

text = bars.mark_text(
    align='center',
    baseline='bottom',
).encode(
    text='count()'
)

(bars + text).interactive().properties(
    height=300, 
    width=700,
    title = "Number of articles in each category",
)

In [ ]:
df['id'] = 1
df2 = pd.DataFrame(df.groupby('Category').count()['id']).reset_index()

bars = alt.Chart(df2).mark_bar(size=50).encode(
    x=alt.X('Category'),
    y=alt.Y('PercentOfTotal:Q', axis=alt.Axis(format='.0%', title='% of Articles')),
    color='Category'
).transform_window(
    TotalArticles='sum(id)',
    frame=[None, None]
).transform_calculate(
    PercentOfTotal="datum.id / datum.TotalArticles"
)

text = bars.mark_text(
    align='center',
    baseline='bottom',
    #dx=5  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('PercentOfTotal:Q', format='.1%')
)

(bars + text).interactive().properties(
    height=300, 
    width=700,
    title = "% of articles in each category",
)

In [ ]:
df['News_length'] = df['Content'].str.len()

In [ ]:
plt.figure(figsize=(12.8,6))
sns.distplot(df['News_length']).set_title('News length distribution');

In [ ]:
df['News_length'].describe()

In [ ]:
quantile_95 = df['News_length'].quantile(0.95)
df_95 = df[df['News_length'] < quantile_95]

In [ ]:
plt.figure(figsize=(12.8,6))
sns.distplot(df_95['News_length']).set_title('News length distribution');

In [ ]:
df_more10k = df[df['News_length'] > 10000]
len(df_more10k)

In [ ]:
df_more10k['Content'].iloc[0]

In [ ]:
plt.figure(figsize=(12.8,6))
sns.boxplot(data=df, x='Category', y='News_length', width=.5);

In [ ]:
plt.figure(figsize=(12.8,6))
sns.boxplot(data=df_95, x='Category', y='News_length');

In [ ]:
with open('News_dataset.pickle', 'wb') as output:
    pickle.dump(df, output)

In [1]:

import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [3]:
path_df = "C:\\FE\\autocar_dataset.csv"
df = pd.read_csv(path_df)
#with open(path_df, 'rb') as data:
 #   df = pickle.load(data)

In [4]:
df.head()

,Unnamed: 0,Content,Category,Complete_Filename,File_Name
0,0,"Imagine getting into your car, typing—or, bett...",autonomous car,Investopedia,NaN
1,1,"In 2009, Google started the self-driving car p...",autonomous car,Investopedia,NaN
2,2,"Since then, Waymo has invited the public to jo...",autonomous car,Investopedia,NaN
3,3,“The building blocks of driverless cars are on...,autonomous car,Investopedia,NaN
4,4,These systems were quickly followed by technol...,autonomous car,Investopedia,NaN


In [5]:
df.loc[1]['Content']

"In 2009, Google started the self-driving car project with the goal of driving autonomously over ten uninterrupted 100-mile routes. In 2016, Waymo, an autonomous driving technology company, became a subsidiary of Alphabet, and Google's self-driving project became Waymo."

In [6]:
df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
text = "Mr Greenspan\'s"
text

"Mr Greenspan's"

In [7]:
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")


<ipython-input-7-40211d7fa942>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')


In [8]:
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

------------------------------------------------------------


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pxy04\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pxy04\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
wordnet_lemmatizer = WordNetLemmatizer()

nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

df['Content_Parsed_5'] = lemmatized_text_list
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pxy04\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
stop_words = list(stopwords.words('english'))

stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [11]:
example = "me eating a meal"
word = "me"

# The regular expression is:
regex = r"\b" + word + r"\b"  # we need to build it like that to work properly

re.sub(regex, "StopWord", example)

'StopWord eating a meal'

In [12]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

<ipython-input-12-3b7196a1b53b>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')


In [13]:
df.loc[5]['Content']

'In 2018, Waymo announced that they would be making self-driving cars available by 2020. However, despite some extraordinary advancements, in the year 2020, self-driving cars are still out of reach, except in some trial programs.\ufeff\ufeff The current technology on the market is limited to cars that will automatically brake for you if they anticipate a collision, cars that help keep you in your lane, and cars that can mostly handle highway driving.'

In [14]:
df.loc[5]['Content_Parsed_1']
df.loc[5]['Content_Parsed_2']
df.loc[5]['Content_Parsed_3']
df.loc[5]['Content_Parsed_4']
df.loc[5]['Content_Parsed_5']
df.loc[5]['Content_Parsed_6']

' 2018 waymo announce   would  make self-driving cars available  2020 however despite  extraordinary advancements   year 2020 self-driving cars  still   reach except   trial programs\ufeff\ufeff  current technology   market  limit  cars   automatically brake     anticipate  collision cars  help keep    lane  cars   mostly handle highway drive'

In [15]:

df.head(1)

,Unnamed: 0,Content,Category,Complete_Filename,File_Name,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,0,"Imagine getting into your car, typing—or, bett...",autonomous car,Investopedia,NaN,"Imagine getting into your car, typing—or, bett...","imagine getting into your car, typing—or, bett...",imagine getting into your car typing—or better...,imagine getting into your car typing—or better...,imagine get into your car typing—or better yet...,imagine get car typing— better yet speaking—...


In [16]:
list_columns = ["File_Name", "Category", "Complete_Filename", "Content", "Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

In [17]:
df.head()

,File_Name,Category,Complete_Filename,Content,Content_Parsed
0,NaN,autonomous car,Investopedia,"Imagine getting into your car, typing—or, bett...",imagine get car typing— better yet speaking—...
1,NaN,autonomous car,Investopedia,"In 2009, Google started the self-driving car p...",2009 google start self-driving car project ...
2,NaN,autonomous car,Investopedia,"Since then, Waymo has invited the public to jo...",since waymo invite public join first publ...
3,NaN,autonomous car,Investopedia,“The building blocks of driverless cars are on...,“ build block driverless cars road ” expla...
4,NaN,autonomous car,Investopedia,These systems were quickly followed by technol...,systems quickly follow technology allow ca...


In [18]:
category_codes = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4
}
df['Category_Code'] = df['Category']
df = df.replace({'Category_Code':category_codes})
df.head()

,File_Name,Category,Complete_Filename,Content,Content_Parsed,Category_Code
0,NaN,autonomous car,Investopedia,"Imagine getting into your car, typing—or, bett...",imagine get car typing— better yet speaking—...,autonomous car
1,NaN,autonomous car,Investopedia,"In 2009, Google started the self-driving car p...",2009 google start self-driving car project ...,autonomous car
2,NaN,autonomous car,Investopedia,"Since then, Waymo has invited the public to jo...",since waymo invite public join first publ...,autonomous car
3,NaN,autonomous car,Investopedia,“The building blocks of driverless cars are on...,“ build block driverless cars road ” expla...,autonomous car
4,NaN,autonomous car,Investopedia,These systems were quickly followed by technol...,systems quickly follow technology allow ca...,autonomous car


In [30]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300


2131    microsoft set sight  spyware  windows users co...
1518    palace threat  cantona mask  manchester unite ...
1996    pompeii get digital make-   old-fashioned audi...
850     prodigy join v festival line-  essex act prodi...
1515     injury mar uefa victory  middlesbrough boss s...
                              ...                        
986     protesters plan airport challenge  campaigners...
2181    apple mac mini get warm welcome   mac mini   w...
2033    new year texting break record   mobile phone  ...
1364    howard unfinished business    finish yet whisp...
451     us airways staff agree  pay cut   union repres...
Name: Content_Parsed, Length: 1940, dtype: object

In [31]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(1940, 300)
(343, 300)


In [32]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'business' category:
  . Most correlated unigrams:
. economic
. economy
. oil
. growth
. bank
  . Most correlated bigrams:
. last year
. year old

# 'entertainment' category:
  . Most correlated unigrams:
. tv
. music
. star
. award
. film
  . Most correlated bigrams:
. mr blair
. prime minister

# 'politics' category:
  . Most correlated unigrams:
. minister
. party
. blair
. election
. labour
  . Most correlated bigrams:
. prime minister
. mr blair

# 'sport' category:
  . Most correlated unigrams:
. club
. game
. win
. side
. match
  . Most correlated bigrams:
. say mr
. year old

# 'tech' category:
  . Most correlated unigrams:
. technology
. digital
. computer
. software
. users
  . Most correlated bigrams:
. year old
. say mr



In [33]:
bigrams

['tell bbc', 'last year', 'prime minister', 'mr blair', 'year old', 'say mr']

In [23]:
# X_train
#with open('C:\\FE\\X_train.pickle', 'wb') as output:
 #   pickle.dump(X_train, output)
    
# X_test    ##with open('C:\\FE\\X_test.pickle', 'wb') as output:
 #   pickle.dump(X_test, output)
    
# y_train
#with open('C:\\FE\\y_train.pickle', 'wb') as output:
 #   pickle.dump(y_train, output)
    
# y_test
#with open('C:\\FE\\y_test.pickle', 'wb') as output:
 #   pickle.dump(y_test, output)
    
# df
##with open('C:\\FE\\df.pickle', 'wb') as output:
 #   pickle.dump(df, output)
    
# features_train
#with open('C:\\FE\\features_train.pickle', 'wb') as output:
 #   pickle.dump(features_train, output)
#
## labels_train
#with open('C:\\FE\\labels_train.pickle', 'wb') as output:
 #   pickle.dump(labels_train, output)

# features_test
#with open('C:\\FE\\features_test.pickle', 'wb') as output:
 #   pickle.dump(features_test, output)

# labels_test
#with open('C:\\FE\\labels_test.pickle', 'wb') as output:
 #   pickle.dump(labels_test, output)
    
# TF-IDF object
#with open('C:\\FE\\tfidf.pickle', 'wb') as output:
 #   pickle.dump(tfidf, output)

In [34]:

import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [35]:

# Dataframe
path_df = "C:\\FE\\df.pickle"
with open(path_df, 'rb') as data:
    df = pickle.load(data)

# features_train
path_features_train = "C:\\FE\\features_train.pickle"
with open(path_features_train, 'rb') as data:
    features_train = pickle.load(data)

# labels_train
path_labels_train = "C:\\FE\\labels_train.pickle"
with open(path_labels_train, 'rb') as data:
    labels_train = pickle.load(data)

# features_test
path_features_test = "C:\\FE\\features_test.pickle"
with open(path_features_test, 'rb') as data:
    features_test = pickle.load(data)

# labels_test
path_labels_test = "C:\\FE\\labels_test.pickle"
with open(path_labels_test, 'rb') as data:
    labels_test = pickle.load(data)

In [36]:
print(features_train.shape)
print(features_test.shape)

(1940, 300)
(343, 300)


In [37]:
rf_0 = RandomForestClassifier(random_state = 8)

print('Parameters currently in use:\n')
pprint(rf_0.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 8,
 'verbose': 0,
 'warm_start': False}


In [38]:
# n_estimators
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 5)]

# max_features
max_features = ['auto', 'sqrt']

# max_depth
max_depth = [int(x) for x in np.linspace(20, 100, num = 5)]
max_depth.append(None)

# min_samples_split
min_samples_split = [2, 5, 10]

# min_samples_leaf
min_samples_leaf = [1, 2, 4]

# bootstrap
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [20, 40, 60, 80, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000]}


In [39]:
# First create the base model to tune
rfc = RandomForestClassifier(random_state=8)

# Definition of the random search
random_search = RandomizedSearchCV(estimator=rfc,
                                   param_distributions=random_grid,
                                   n_iter=50,
                                   scoring='accuracy',
                                   cv=3, 
                                   verbose=1, 
                                   random_state=8)

# Fit the random search model
random_search.fit(features_train, labels_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


C:\Users\pxy04\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\pxy04\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\pxy04\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 331, in fit
    y, expanded_class_weight = self._validate_y_class_weight(y)
  File "C:\Users\pxy04\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 559, in _validate_y_class_weight
    check_classification_targets(y)
  File "C:\Users\pxy04\Anaconda3\lib\site-packages\sklearn\utils\multiclass.py", line 183, in check_classification_targets
    raise ValueError("Unknown label type: %r" % y_type)
ValueError: Unknown label type: 'unknown'

  warnings.warn("Estimator fit failed. 

ValueError: Unknown label type: 'unknown'

In [40]:

print("The best hyperparameters from Random Search are:")
print(random_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(random_search.best_score_)

The best hyperparameters from Random Search are:
{'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': True}

The mean accuracy of a model with these hyperparameters is:
nan


In [41]:
bootstrap = [False]
max_depth = [30, 40, 50]
max_features = ['sqrt']
min_samples_leaf = [1, 2, 4]
min_samples_split = [5, 10, 15]
n_estimators = [800]

param_grid = {
    'bootstrap': bootstrap,
    'max_depth': max_depth,
    'max_features': max_features,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'n_estimators': n_estimators
}

# Create a base model
rfc = RandomForestClassifier(random_state=8)

# Manually create the splits in CV in order to be able to fix a random_state (GridSearchCV doesn't have that argument)
cv_sets = ShuffleSplit(n_splits = 3, test_size = .33, random_state = 8)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rfc, 
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=cv_sets,
                           verbose=1)

# Fit the grid search to the data
grid_search.fit(features_train, labels_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


C:\Users\pxy04\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\pxy04\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\pxy04\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 331, in fit
    y, expanded_class_weight = self._validate_y_class_weight(y)
  File "C:\Users\pxy04\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 559, in _validate_y_class_weight
    check_classification_targets(y)
  File "C:\Users\pxy04\Anaconda3\lib\site-packages\sklearn\utils\multiclass.py", line 183, in check_classification_targets
    raise ValueError("Unknown label type: %r" % y_type)
ValueError: Unknown label type: 'unknown'

  warnings.warn("Estimator fit failed. 

ValueError: Unknown label type: 'unknown'

In [42]:
print("The best hyperparameters from Grid Search are:")
print(grid_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(grid_search.best_score_)

The best hyperparameters from Grid Search are:
{'bootstrap': False, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 800}

The mean accuracy of a model with these hyperparameters is:
nan


In [43]:

best_rfc = grid_search.best_estimator_
best_rfc

RandomForestClassifier(bootstrap=False, max_depth=30, max_features='sqrt',
                       min_samples_split=5, n_estimators=800, random_state=8)

In [44]:
best_rfc.fit(features_train, labels_train)
rfc_pred = best_rfc.predict(features_test)

print("The training accuracy is: ")
print(accuracy_score(labels_train, best_rfc.predict(features_train)))
print("The test accuracy is: ")
print(accuracy_score(labels_test, rfc_pred))

print("Classification report")
print(classification_report(labels_test,rfc_pred))

aux_df = df[['Category', 'Category_Code']].drop_duplicates().sort_values('Category_Code')
conf_matrix = confusion_matrix(labels_test, rfc_pred)
plt.figure(figsize=(12.8,6))
sns.heatmap(conf_matrix, 
            annot=True,
            xticklabels=aux_df['Category'].values, 
            yticklabels=aux_df['Category'].values,
            cmap="Blues")
plt.ylabel('Predicted')
plt.xlabel('Actual')
plt.title('Confusion matrix')
plt.show()

base_model = RandomForestClassifier(random_state = 8)
base_model.fit(features_train, labels_train)
accuracy_score(labels_test, base_model.predict(features_test))

best_rfc.fit(features_train, labels_train)
accuracy_score(labels_test, best_rfc.predict(features_test))

d = {
     'Model': 'Random Forest',
     'Training Set Accuracy': accuracy_score(labels_train, best_rfc.predict(features_train)),
     'Test Set Accuracy': accuracy_score(labels_test, rfc_pred)
}

df_models_rfc = pd.DataFrame(d, index=[0])

df_models_rfc




ValueError: Unknown label type: 'unknown'